In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('spam.csv',encoding='latin-1')
# Keeping only the neccessary columns
data = data[['v2','v1']]

data['v2'] = data['v2'].apply(lambda x: x.lower())
data['v2'] = data['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

print(data[data['v1'] == 'ham'].size)
print(data[data['v1'] == 'spam'].size)

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['v2'].values)
X = tokenizer.texts_to_sequences(data['v2'].values)
print(X)
X = pad_sequences(X)
print(X)
embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

model = createmodel()
model.fit(X_train, Y_train, epochs = 4, batch_size=40, verbose = 2)

score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=40)
print(score)
print(acc)

Using TensorFlow backend.


9650
1494
[[46, 442, 794, 711, 680, 64, 9, 1245, 90, 119, 356, 1246, 154, 1247, 67, 58, 137], [48, 312, 1399, 443, 6, 1823], [50, 459, 9, 22, 4, 749, 899, 1, 179, 1824, 1131, 625, 1825, 261, 71, 1824, 1, 1826, 1, 313, 459, 79, 382], [6, 229, 142, 24, 357, 6, 160, 143, 60, 142], [949, 2, 43, 97, 72, 460, 1, 900, 72, 1827, 202, 111, 461], [844, 120, 67, 1588, 41, 100, 184, 597, 21, 7, 39, 343, 86, 358, 56, 109, 383, 3, 42, 12, 15, 84, 1828, 48, 369, 1034, 1, 70, 302, 1], [203, 11, 598, 8, 25, 56, 1, 359, 35, 10, 108, 681, 10, 56], [73, 216, 13, 1132, 1400, 1400, 1829, 112, 100, 573, 73, 13, 950, 12, 51, 1589, 951, 462, 1, 1035, 13, 217, 950], [682, 73, 4, 795, 413, 218, 3, 17, 100, 414, 1, 1036, 140, 952, 1, 115, 16, 115, 401, 504, 712, 548, 64], [129, 13, 92, 1037, 796, 27, 124, 6, 85, 1133, 1, 463, 1, 5, 303, 525, 845, 35, 329, 12, 50, 16, 5, 92, 463, 1248, 50, 18], [23, 219, 32, 80, 209, 7, 2, 43, 69, 1, 264, 77, 40, 276, 1134, 210, 180, 168, 415, 91], [1830, 1, 179, 173, 47, 314, 1, 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
 - 45s - loss: 0.2062 - accuracy: 0.9258
Epoch 2/4
 - 44s - loss: 0.0496 - accuracy: 0.9855
Epoch 3/4
 - 44s - loss: 0.0312 - accuracy: 0.9893
Epoch 4/4
 - 44s - loss: 0.0205 - accuracy: 0.9928
0.07303126994267212
0.9809679388999939
